# # Portfolio Optimization Experiments
# 
# This notebook demonstrates how to use the configuration system and validation pipeline.


In [7]:
!pip install pyyaml
!pip install pandas
!pip install yfinance
!pip install numpy
!pip install torch

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 9.7 MB/s  0:00:01.8 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 9.5 MB/s  0:00:00m 9.7 MB/s eta 0:00:01
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [pandas]━━━━ 3/4 [pandas]
  Using cached yfinance-0.2.66-py2.py3-none-any.whl.metadata (6.0 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached multitasking-0.0.12-py3-none-any.whl
  Using cached frozendict-2.4.7-py3-none-any.whl.metadata (23 kB)
  Using cached peewee-3.18.3.tar.gz (3.0 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached curl_cffi-0.14.0-cp39-abi3-macosx_14_0_arm64.whl.metadata (15 kB)
  Using cached protobuf-6.33.2-cp39-abi3-macosx_10_9_universal2.whl.metadata (593 bytes)
  Using ca

In [1]:
import sys
sys.path.append('config')

from config_manager import (
    Config, 
    quick_test_config, 
    full_experiment_config,
    DDPG_SEARCH_SPACE,
    DIV_DDPG_SEARCH_SPACE,
    PGA_SEARCH_SPACE
)
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

# ## 1. Configuration System
# 
# Load defaults and override as needed. All defaults are from original papers.

# %%
# Option A: Load defaults, no changes

In [2]:
cfg = Config()
print(f"Default DDPG learning rate: {cfg.ddpg.actor_lr}")
print(f"Default validation strategy: {cfg.validation.strategy}")


Default DDPG learning rate: 0.0001
Default validation strategy: expanding_window


In [ ]:
cfg = Config(
    ddpg={
        'actor_lr': 0.0005,  # changed from 0.0001
        'total_timesteps': 100000  # reduced for faster testing
    },
    validation={
        'strategy': 'holdout'  # simpler validation
    }
)

In [ ]:
cfg.save('experiments/run_001/config.yaml')


# ## 2. Validation Strategies Explained
#
# ### Expanding Window (Recommended for thesis)
# 
# ```
# Year:  05  06  07  08  09  10  11  12  13  14  15  16  17  18  19  20  21  22  23  24
#        |----------TRAIN----------|--VAL--|                              |--TEST--|
#        |-------------TRAIN---------------|--VAL--|                      |--TEST--|
#        |------------------TRAIN------------------|--VAL--|              |--TEST--|
#        ...
# ```
# 
# - Training window grows each fold
# - Validation window slides forward  
# - Test set is held out entirely until final evaluation
# - Gives multiple performance estimates across market regimes


In [3]:
def create_expanding_window_splits(
    data: pd.DataFrame,
    initial_train_years: int = 10,
    val_window_years: int = 1,
    step_years: int = 1,
    final_test_years: int = 2,
    purge_gap_days: int = 5
):
    """
    Create expanding window splits for time series cross-validation.
    
    Returns list of (train_idx, val_idx) tuples plus final test_idx.
    """
    trading_days_per_year = 252
    
    # Calculate indices
    n_total = len(data)
    test_size = final_test_years * trading_days_per_year
    test_start = n_total - test_size
    
    initial_train_size = initial_train_years * trading_days_per_year
    val_size = val_window_years * trading_days_per_year
    step_size = step_years * trading_days_per_year
    
    splits = []
    train_end = initial_train_size
    
    while train_end + purge_gap_days + val_size <= test_start:
        train_idx = np.arange(0, train_end)
        val_start = train_end + purge_gap_days
        val_end = val_start + val_size
        val_idx = np.arange(val_start, min(val_end, test_start))
        
        if len(val_idx) > 0:
            splits.append((train_idx, val_idx))
        
        train_end += step_size
    
    test_idx = np.arange(test_start, n_total)
    
    return splits, test_idx


In [4]:
dates = pd.date_range('2005-01-01', '2024-12-31', freq='B')  # business days
dummy_data = pd.DataFrame(index=dates[:5000])  # ~20 years

splits, test_idx = create_expanding_window_splits(
    dummy_data,
    initial_train_years=10,
    val_window_years=1,
    final_test_years=2
)

print(f"Number of validation folds: {len(splits)}")
print(f"Test set size: {len(test_idx)} days")

for i, (train_idx, val_idx) in enumerate(splits[:3]):
    print(f"Fold {i+1}: train={len(train_idx)} days, val={len(val_idx)} days")


Number of validation folds: 7
Test set size: 504 days
Fold 1: train=2520 days, val=252 days
Fold 2: train=2772 days, val=252 days
Fold 3: train=3024 days, val=252 days


## 3. Experiment Runner Structure


In [5]:
class ExperimentRunner:
    """
    Runs experiments with proper validation and logging.
    """
    
    def __init__(self, config: Config):
        self.cfg = config
        self.results = {}
        
    def run_method(self, method: str, train_data, val_data, seed: int):
        """
        Train and evaluate a single method on given data split.
        
        Parameters
        ----------
        method : str
            One of 'equal_weights', 'ddpg', 'div_ddpg', 'pga_map_elites'
        train_data : pd.DataFrame
            Training data (returns)
        val_data : pd.DataFrame
            Validation data (returns)
        seed : int
            Random seed for reproducibility
        """
        # Placeholder - actual implementations would go here
        if method == 'equal_weights':
            # No training needed
            return self._evaluate_equal_weights(val_data)
        
        elif method == 'ddpg':
            cfg = self.cfg.ddpg
            # agent = DDPGAgent(
            #     state_dim=train_data.shape[1] * cfg.lookback_window,
            #     action_dim=train_data.shape[1],
            #     actor_lr=cfg.actor_lr,
            #     critic_lr=cfg.critic_lr,
            #     ...
            # )
            # agent.train(train_data, cfg.total_timesteps)
            # return self._evaluate_agent(agent, val_data)
            pass
        
        elif method == 'div_ddpg':
            cfg = self.cfg.div_ddpg
            # Similar to DDPG but with diversity loss
            pass
        
        elif method == 'pga_map_elites':
            cfg = self.cfg.pga_map_elites
            # Returns archive of policies, need to select representative
            pass
    
    def run_full_comparison(self, data: pd.DataFrame):
        """
        Run all methods across all validation folds and seeds.
        """
        methods = ['equal_weights', 'ddpg', 'div_ddpg', 'pga_map_elites']
        seeds = self.cfg.validation.random_seeds
        
        # Create validation splits
        splits, test_idx = create_expanding_window_splits(
            data,
            initial_train_years=self.cfg.validation.temporal.initial_train_years,
            val_window_years=self.cfg.validation.temporal.val_window_years,
            final_test_years=self.cfg.validation.temporal.final_test_years,
            purge_gap_days=self.cfg.validation.temporal.purge_gap_days
        )
        
        results = {method: [] for method in methods}
        
        for fold_idx, (train_idx, val_idx) in enumerate(splits):
            train_data = data.iloc[train_idx]
            val_data = data.iloc[val_idx]
            
            for method in methods:
                for seed in seeds:
                    print(f"Fold {fold_idx+1}, {method}, seed {seed}")
                    
                    # Run experiment
                    fold_results = self.run_method(
                        method, train_data, val_data, seed
                    )
                    
                    fold_results['fold'] = fold_idx
                    fold_results['seed'] = seed
                    results[method].append(fold_results)
        
        return results
    
    def _evaluate_equal_weights(self, data):
        """Evaluate 1/N portfolio."""
        n_assets = data.shape[1]
        weights = np.ones(n_assets) / n_assets
        returns = (data * weights).sum(axis=1)
        
        return {
            'cumulative_return': (1 + returns).prod() - 1,
            'sharpe_ratio': returns.mean() / returns.std() * np.sqrt(252),
            'max_drawdown': self._max_drawdown(returns),
        }
    
    def _max_drawdown(self, returns):
        """Calculate maximum drawdown."""
        cum_returns = (1 + returns).cumprod()
        rolling_max = cum_returns.expanding().max()
        drawdowns = cum_returns / rolling_max - 1
        return drawdowns.min()


## 4. Running Experiments from Notebook


In [ ]:
# Quick test run
cfg = quick_test_config()

# Override anything you want to test
#cfg.set('ddpg.actor_lr', 0.0003)
#cfg.set('validation.n_seeds', 2)

print("Running with config:")
print(f"  DDPG lr: {cfg.ddpg.actor_lr}")
print(f"  Seeds: {cfg.validation.n_seeds}")
print(f"  Timesteps: {cfg.ddpg.total_timesteps}")

# runner = ExperimentRunner(cfg)
# results = runner.run_full_comparison(data)

# ## 5. Hyperparameter Search with Optuna

In [ ]:
def create_optuna_study(method: str, cfg: Config):
    """
    Create Optuna study for hyperparameter optimization.
    """
    import optuna
    
    search_space = {
        'ddpg': DDPG_SEARCH_SPACE,
        'div_ddpg': DIV_DDPG_SEARCH_SPACE,
        'pga_map_elites': PGA_SEARCH_SPACE
    }[method]
    
    def objective(trial):
        # Sample hyperparameters
        overrides = {}
        for param, spec in search_space.items():
            if spec['type'] == 'loguniform':
                value = trial.suggest_float(param, spec['low'], spec['high'], log=True)
            elif spec['type'] == 'uniform':
                value = trial.suggest_float(param, spec['low'], spec['high'])
            elif spec['type'] == 'categorical':
                value = trial.suggest_categorical(param, spec['choices'])
            
            # Handle nested params like 'diversity.alpha_initial'
            keys = param.split('.')
            d = overrides
            for k in keys[:-1]:
                d = d.setdefault(k, {})
            d[keys[-1]] = value
        
        # Create config with sampled hyperparameters
        trial_cfg = Config(**{method: overrides})
        
        # Run experiment and return validation metric
        # runner = ExperimentRunner(trial_cfg)
        # results = runner.run_method(method, train_data, val_data, seed=42)
        # return results['sharpe_ratio']
        
        return np.random.random()  # placeholder
    
    study = optuna.create_study(direction='maximize')
    return study, objective

# Usage:
# study, objective = create_optuna_study('ddpg', cfg)
# study.optimize(objective, n_trials=50)
# best_params = study.best_params